In [341]:
import os
os.getcwd()

In [342]:
import pandas as pd
df=pd.read_csv('train_set_airbnb.csv')

In [343]:
df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,6/28/10,2.010000e+13,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,5/25/11,2.010000e+13,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,9/28/10,2.010000e+13,8/2/10,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,12/5/11,2.010000e+13,9/8/12,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,9/14/10,2.010000e+13,2/18/10,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [344]:
len(df)

213451

In [345]:
#Since including age < 16 and > 97 does not make sense so let us delete the records with such values


df = df.drop(df[(df['age'] < 16) | (df['age'] > 95)].index)

In [346]:
len(df)

210948

In [347]:
df.columns[df.isnull().any()]

Index(['date_first_booking', 'age', 'first_affiliate_tracked'], dtype='object')

In [348]:
#The idea is to find the difference between the two dates: date_account_created and date_first_booking so will understand
#how long customers took to book a property

import datetime
df['date_diff']=abs(pd.to_datetime(df['date_account_created'], format='%m/%d/%y') - pd.to_datetime(df['date_first_booking'],format='%m/%d/%y')) 

In [349]:
df['date_diff']=df['date_diff'].dt.days

In [350]:
df['date_diff'].max()

365.0

In [351]:
df['date_diff'].fillna(-(df['date_diff'].max()), inplace=True)

In [352]:
df['date_diff'].isnull().any()

False

In [353]:
# import numpy as np
# tfa = np.vstack(df.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
# df['tfa_year'] = tfa[:,0]
# df['tfa_month'] = tfa[:,1]
# df['tfa_day'] = tfa[:,2]
# df_all = df_all.drop(['timestamp_first_active'], axis=1)

In [355]:
df['age'].median()

34.0

In [356]:
df['age'].mean()

36.514069844987716

In [357]:
df['age'].std()

11.570836687369358

In [358]:
#Let's repalce the null values in df['age'] with numbers ranging between one standard deviation from the mean, i.e., 
#from 25 to 47

import random
df['age'].fillna(np.random.choice(range(25, 47)), inplace=True)

In [360]:
dict(df['gender'].value_counts()/len(df))

{'-unknown-': 0.4501630733640518,
 'FEMALE': 0.29402032728444927,
 'MALE': 0.2544939985209625,
 'OTHER': 0.0013226008305364356}

In [361]:
df['gender'].value_counts()

-unknown-    94961
FEMALE       62023
MALE         53685
OTHER          279
Name: gender, dtype: int64

In [362]:
def convert_gender(d, x, y, col_name):
    import pandas as pd
    
    for ele in range(len(x)):
        if (x[ele] == 'FEMALE') | (x[ele] == 'MALE') | (x[ele] == 'OTHER'):
            if x[ele] in y:
                x[ele]=y[x[ele]]
        
        elif x[ele] =='-unknown-':
            x[ele]=0
        
    d.drop(col_name, axis=1)
    d[col_name]=pd.Series(x, index=d.index)
    return d[col_name].head()

convert_gender(df, df['gender'].tolist(), dict(df['gender'].value_counts()/len(df)), 'gender')


0    0.000000
1    0.254494
2    0.294020
3    0.294020
4    0.000000
Name: gender, dtype: float64

In [363]:
df['gender'].value_counts()

0.000000    94961
0.294020    62023
0.254494    53685
0.001323      279
Name: gender, dtype: int64

In [364]:
#To convert the categorical values in the column: country_destination

def convert_country(d, x, col_name):
    import pandas as pd
    
    for ele in range(len(x)):
        if x[ele]=='NDF':
            x[ele]=0
        
        elif x[ele]=='US':
            x[ele]=-1
        
        else:
            x[ele]=1
          
    d.drop(col_name, axis=1)
    d[col_name]=pd.Series(x, index=d.index)
    return d[col_name].head()      
            
            
convert_country(df, df['country_destination'].tolist(), 'country_destination')            
    

0    0
1    0
2   -1
3    1
4   -1
Name: country_destination, dtype: int64

In [365]:
df['country_destination'].isnull().any()

False

In [366]:
#To convert columns: 'signup_method', 'signup_flow','language', 'affiliate_channel', 'affiliate_provider',
#'first_affiliate_tracked', 'signup_app', 'first_device_type','first_browser'

def convert_categorical(d, x, y, col_name):
    import pandas as pd
    for ele in range(len(x)):
        if x[ele] in y:
            x[ele]=y[x[ele]]
         
    d.drop(col_name, axis=1)
    d[col_name]=pd.Series(x, index=d.index)
    return d[col_name].head()

convert_categorical(df, df['signup_method'].tolist(), dict(df['signup_method'].value_counts()/len(df)), 'signup_method')
convert_categorical(df, df['signup_flow'].tolist(), dict(df['signup_flow'].value_counts()/len(df)), 'signup_flow')
convert_categorical(df, df['language'].tolist(), dict(df['language'].value_counts()/len(df)), 'language')
convert_categorical(df, df['affiliate_channel'].tolist(), dict(df['affiliate_channel'].value_counts()/len(df)), 'affiliate_channel')
convert_categorical(df, df['affiliate_provider'].tolist(), dict(df['affiliate_provider'].value_counts()/len(df)), 'affiliate_provider')
convert_categorical(df, df['first_affiliate_tracked'].tolist(), dict(df['first_affiliate_tracked'].value_counts()/len(df)), 'first_affiliate_tracked')
convert_categorical(df, df['signup_app'].tolist(), dict(df['signup_app'].value_counts()/len(df)), 'signup_app')
convert_categorical(df, df['first_device_type'].tolist(), dict(df['first_device_type'].value_counts()/len(df)), 'first_device_type')
convert_categorical(df, df['first_browser'].tolist(), dict(df['first_browser'].value_counts()/len(df)), 'first_browser')




















0    0.299206
1    0.299206
2    0.098493
3    0.157499
4    0.299206
Name: first_browser, dtype: float64

In [ ]:
#looks like df['first_affiliate_tracked'] has null values so avoiding it for now

In [367]:
df['affiliate_channel'].isnull().any()

False

In [368]:
df.columns[df.isnull().any()]

Index(['date_first_booking', 'first_affiliate_tracked'], dtype='object')

In [369]:
df.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination', 'date_diff'],
      dtype='object')

In [382]:
cols = [col for col in df.columns if col in ['id','gender', 'age', 'signup_method','signup_flow','language', 'affiliate_channel', 'affiliate_provider', 'signup_app', 'first_device_type','first_browser','date_diff', 'country_destination']]
df1 = df[cols]

In [383]:
X_id=df1['id'].values

In [384]:
df1=df1.drop('id', axis=1)

In [385]:
df1.head()

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,signup_app,first_device_type,first_browser,country_destination,date_diff
0,0.000000,35.0,0.281524,0.771209,0.966428,0.645415,0.644012,0.855216,0.419246,0.299206,0,-365.0
1,0.254494,38.0,0.281524,0.771209,0.966428,0.040541,0.241922,0.855216,0.419246,0.299206,0,-365.0
2,0.294020,56.0,0.715892,0.041361,0.966428,0.645415,0.644012,0.855216,0.340510,0.098493,-1,57.0
3,0.294020,42.0,0.281524,0.771209,0.966428,0.645415,0.644012,0.855216,0.419246,0.157499,1,278.0
4,0.000000,41.0,0.715892,0.771209,0.966428,0.645415,0.644012,0.855216,0.419246,0.299206,-1,208.0


In [387]:
y=df1['country_destination'].values

In [403]:
len(np.vstack(y))

210948

In [388]:
df1=df1.drop('country_destination', axis=1)

In [399]:
X=np.vstack(df1.values)

In [408]:
len(X)

210948

In [409]:
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)
# #keep X_valid and y_valid separately

In [410]:
from sklearn.model_selection import StratifiedKFold
folds = 10
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 999)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [405]:
#Using LogisticRegression Classifier with default values
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

In [412]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
print(scores)

[0.87612577 0.87612577 0.87612577 0.87611925 0.87616539 0.87616539
 0.87616539 0.87616539 0.87615887 0.87615887]


In [416]:
#
# clf1 = LogisticRegression(penalty='l1', C=0.001, random_state=0, solver= 'saga', multi_class='multinomial')
# clf2 = LogisticRegression(penalty='l1', C=0.01, random_state=0, solver='lbfgs', multi_class='multinomial')
# clf3 = LogisticRegression(penalty='l1', C=10, random_state=0, solver='lbfgs', multi_class='multinomial')
# clf4 = LogisticRegression(penalty='l1', C=100, random_state=0, solver='lbfgs', multi_class='multinomial')

In [417]:
clf5 = LogisticRegression(penalty='l2', C=0.001, random_state=0, solver='lbfgs', multi_class='multinomial')
clf6 = LogisticRegression(penalty='l2', C=0.01, random_state=0, solver='lbfgs', multi_class='multinomial')
clf7 = LogisticRegression(penalty='l2', C=10, random_state=0, solver='lbfgs', multi_class='multinomial')
clf8 = LogisticRegression(penalty='l2', C=100, random_state=0, solver='lbfgs', multi_class='multinomial')

In [419]:
#clf5
scores = cross_val_score(clf5, X_train, y_train, cv=10, scoring='accuracy')
print(scores)

[0.87612577 0.87612577 0.87612577 0.87611925 0.87616539 0.87616539
 0.87616539 0.87616539 0.87615887 0.87615887]


In [420]:
#clf6
scores = cross_val_score(clf6, X_train, y_train, cv=10, scoring='accuracy')
print(scores)

[0.87612577 0.87612577 0.87612577 0.87611925 0.87616539 0.87616539
 0.87616539 0.87616539 0.87615887 0.87615887]


In [421]:
#clf7
scores = cross_val_score(clf7, X_train, y_train, cv=10, scoring='accuracy')
print(scores)

[0.87612577 0.87612577 0.87612577 0.87611925 0.87616539 0.87616539
 0.87616539 0.87616539 0.87615887 0.87615887]


In [424]:
from sklearn.model_selection import cross_val_predict
cross_val_predict(clf, X_test, y_test, cv=10)

array([-1, -1,  0, ...,  0,  0,  0])

In [425]:
base_model=clf.fit(X_train, y_train)

In [428]:
y_predict=base_model.predict(X_test)

In [431]:
predictions=base_model.predict_proba(X_test)

In [438]:
#Cannot find roc_auc_score for multiclass problems
# from sklearn import metrics
# auc = metrics.roc_auc_score(y_test, predictions)

In [444]:
# from sklearn.metrics import roc_auc_score
# roc = {label: [] for label in np.unique(y_train)}
# for label in np.unique(y_train):
#     clf.fit(X_train, y_train == label)
#     predictions_proba = clf.predict_proba(X_test)
#     roc[label] += roc_auc_score(y_test, predictions_proba[:,label])
    


In [440]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predict, average='weighted')  

/Users/chidam/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.8251081357619214

In [443]:
# from sklearn.metrics import precision_recall_curve
# import matplotlib.pyplot as plt
# precision, recall, _ = precision_recall_curve(y_test, y_predict)
# # step_kwargs = ({'step': 'post'}
#                if 'step' in signature(plt.fill_between).parameters
#                else {})
# plt.step(recall, precision, color='b', alpha=0.2,
#          where='post')
# plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.show()